#### Task 1
### Naive Bayes Classifier Realization
- [x] написать код, который составляет список уникальных слов по текстовой коллекции
- [x] написать код, который переводит текст в бинарное Bag of Words представление используя словарь из 1
- [x] реализовать наивный байесовский классификатор

*Davletyarov Ildar, 11-808*

In [13]:
import nltk
import ssl

# stab for download nltk packages without SSL
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')


[nltk_data] Downloading package punkt to /Users/ildar/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from nltk.stem import PorterStemmer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
from collections import defaultdict

In [3]:
# load data
DATA_PATH = "/Users/ildar/Desktop/labeledTrainData.tsv"
df = pd.read_csv(DATA_PATH, sep='\t', encoding='utf-8')

In [11]:
df.to_dict('records')[0]

{'id': '5814_8',
 'sentiment': 1,
 'review': "With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />Th

In [16]:
texts = df.review.tolist()
sentiment = df.sentiment.tolist()

In [13]:
def space_tokenize(text):
    words = text.split(' ')
    return words
    
    
def get_vocab(texts):
    d = dict((texts.count(i), i) for i in set(texts))
    texts = list(set(texts))
    return d

In [5]:
class NaiveBayesClassifier:
    def __init__(self, n_classes):
        self.n_classes = n_classes
        self.class_probs = {}
        self.conditional_probabilities = {}
        self._class_freq = defaultdict(lambda: 0)
        self._feat_freq = defaultdict(lambda: 0)
    
    def fit(self, X, y):
        for feature, label in zip(X, y):
            self._class_freq[label] += 1
            for value in feature:
                self._feat_freq[(value, label)] += 1
        
        num_samples = len(X) # normalize
        for k in self._class_freq:
            self._class_freq[k] /= num_samples
            
        for value, label in self._feat_freq:
            self._feat_freq[(value, label)] /= self._class_freq[label]
            
        return self       
        
    def predict(self, X):
        return min(self._class_freq.keys(), key=lambda c : self._calculate_class_freq(X, c))
    
    def _calculate_class_freq(self, X, clss): # formula of naive bayes
        freq = -np.log(self._class_freq[clss]) # ln of truth
        for feat in X:
            freq += -np.log(self._feat_freq.get((feat, clss), 10 ** (-7)))
        return freq

In [8]:
X_train, X_test, y_train, y_test = train_test_split(texts, sentiment)

In [17]:
model = NaiveBayesClassifier(n_classes=2)
model.fit(X_train, y_train)
y_pred = [model.predict(x) for x in X_test]

In [10]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.00      0.50      0.00         2
           1       1.00      0.51      0.67      6248

    accuracy                           0.51      6250
   macro avg       0.50      0.50      0.34      6250
weighted avg       1.00      0.51      0.67      6250



In [11]:
len(y_pred)

6250